# Setup

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import time
import copy 
import numpy as np
import pandas as pd
from sklearn import tree 
from sklearn.model_selection import cross_val_score

import matplotlib.pyplot as plt
from cycler import cycler
color_list = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd',
              '#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf']
plt.rc('axes', prop_cycle=(cycler('color',color_list)))

from LRT import LRT
from LRT import moments
from LRT import figfuns
from LRT import PCTBIN

**Parameter struct:**

In [ ]:
par = LRT.ParStruct()
SAMPLE = '_p100'

**Seed:**

In [ ]:
np.random.seed(1917)

## Load data

In [ ]:
# a. load
data = LRT.dataStruct()
data.logY = np.transpose(np.genfromtxt(f'data/logY{SAMPLE}.csv',delimiter=','))

par.T, par.N = data.logY.shape
print(f'(T,N) = ({par.T},{par.N})')

# b. update par
par.simN = max([par.N,par.simN])
par.k = 5
par.depth = 10
par.k_lead = 30

# Single periods

In [ ]:
# a. settings
ts = [5,10,15,20,25]
n_folds = 5
depth_list = range(5,13)

# b. estimate
for t in ts:
    
    # a. get features and targets
    x,y = LRT.get_features_and_targets(par, data, t)
    
    # b. compute scores
    depth = []
    for i in depth_list:        
        treenow = tree.DecisionTreeRegressor(max_depth=i,min_samples_leaf=100)
        treenow.fit(x,y) 
        scores = cross_val_score(estimator=treenow,X=x,y=y,cv=n_folds,n_jobs=1,scoring=LRT.mse_scorer)
        depth.append(scores.mean())

    # c. plot
    fig,ax = figfuns.new()
    ax.plot(depth_list,depth,'-o',markeredgecolor='none')
    ax.set(ylabel='MSE out of fold',xlabel='depth',title='5-fold cross-validation') 
    
    fig.tight_layout() 
    figfuns.save(fig, ax, f'crossval_depth_t{t}')
    plt.close()

# Full life-cycle

## Split sample

In [ ]:
# a. setup
data_est = LRT.dataStruct()
data_holdout = LRT.dataStruct()

# b. split
I_holdout = np.random.binomial(n=1, p=0.8, size=(par.N,)).astype(np.bool)
data_est.logY = data.logY[:,~I_holdout]
data_holdout.logY = data.logY[:,I_holdout]

**Update par:**

In [ ]:
par_est = copy.deepcopy(par)
par_est.N = data_est.logY.shape[1]

par_holdout = copy.deepcopy(par)
par_holdout.N = data_holdout.logY.shape[1]

## LRT

In [ ]:
models = []
marker_list = ['o','x','d','v','^','<','>']

for i,depth in enumerate([2,5,7,10,12,15]): 

    name_short = f'LRT_depth_{depth:d}'
    name = f'LRT (depth {depth:d})'
    color = color_list[i]
    marker = marker_list[i]

    print(name)
    
    # a. settings
    par_est.k = 5
    par_est.k_lead = 30    
    par_est.depth = depth

    # b. estimate
    model = LRT.estimate(par_est,data_est,name,color)
    model.name_short = name_short
    model.marker = marker
    models.append(model)
    
    print('')

prefmodel = models[-1]

## Clustering

In [ ]:
models_clustering = []
marker_list = ['o','x','d','v','^','<','>']
par_cluster = copy.deepcopy(par_est)

for i,depth in enumerate([2,5,7,10,12,15]): 

    name_short = f'LRT_cluster_depth_{depth:d}'
    name = f'LRT no leads/lags (depth {depth:d})'
    color = color_list[i]
    marker = marker_list[i]
    print(name)
    
    # a. settings
    par_cluster.k = 0
    par_cluster.k_lead = 0    
    par_cluster.depth = depth

    # b. estimate
    model = LRT.estimate(par_cluster,data_est,name,color)
    model.name_short = name_short
    model.marker = marker
    models_clustering.append(model)
    
    print('')

## Simulate

In [ ]:
rng_state = np.random.get_state()

In [ ]:
for model in models:
    model.par.N = data_holdout.logY.shape[1]
    model.par.simN = model.par.N 
    model.data = LRT.simulate(model.par,model,data_holdout,seed=None,rng_state=rng_state)

In [ ]:
for model in models_clustering:
    model.par.N = data_holdout.logY.shape[1]
    model.par.simN = model.par.N 
    model.data = LRT.simulate(model.par,model,data_holdout,seed=None,rng_state=rng_state)

## Compute within-person MSE

In [ ]:
def compare_MSE(model,data):
    
    ydata = data.logY
    ypred = model.data.logY
    MSE = np.mean((ydata - ypred)**2)
    
    print(f'{model.name:16s}: MSE = {MSE:.6f}')

In [ ]:
for model in models: 
    compare_MSE(model,data_holdout)

In [ ]:
for model in models_clustering: 
    compare_MSE(model,data_holdout)

# PCTBIN

## Estimate

In [ ]:
models_PCTBIN = [] 
num_bins_list = [4, 10, 20, 50, 75, 100, 200]
for nb in num_bins_list: 
    model_PCTBIN = PCTBIN.estimate(par_est,data_est,num_bins=nb)
    models_PCTBIN.append(model_PCTBIN)

## Simulate

In [ ]:
for model_PCTBIN in models_PCTBIN: 
    model_PCTBIN.data = PCTBIN.simulate(model_PCTBIN,data_holdout)

# Depth vs. number of leafs

In [ ]:
# a. unpack
depths = [model.par.depth for model in models]
leafs = [np.mean(model.num_leafs) for model in models]
max_leafs = [2.0**model.par.depth for model in models]

# b. figure
fig,ax = figfuns.new()
ax.semilogy(depths,leafs,label='used by estimator',marker='o')
ax.semilogy(depths,max_leafs,label='maximum possible',marker='o')
ax.set_xlabel('depth')
ax.set_ylabel('avg. number of groups in tree per age')

max_leafs_list = max_leafs
max_leafs_list.append(max(leafs))
ax.set_yticks(max_leafs_list)
ax.set_yticklabels([ f'{leafs:4.0f}' for leafs in max_leafs_list])
ax.legend()

fig.tight_layout() 
figfuns.save(fig,ax,name='depth_vs_num_leafs')
plt.close()

# Comparing analytical MISE

## Compute MISE for different depth/num_bins

In [ ]:
def compute_mise_list(model_list,t0,t1,modeltype):
    
    MISES = np.empty((len(model_list),))
    for i,model in enumerate(model_list): 
        MISES[i] = LRT.compute_MISE_analytically(model,data_holdout.logY,t0,t1,modeltype)
        
    return MISES

In [ ]:
# a. setting
t0s = [0,4]
jumps = [1,5,10]

# b. allocate
MISES_PCTBIN = np.empty((len(t0s),len(jumps),len(models_PCTBIN)))
MISES_LRT = np.empty((len(t0s),len(jumps),len(models)))
MISES_LRT_clustering = np.empty((len(t0s),len(jumps),len(models_clustering)))

# c. compute
for i,t0 in enumerate(t0s):
    for j,jump in enumerate(jumps): 
        t1 = t0 + jump
        MISES_PCTBIN[i,j,:] = compute_mise_list(models_PCTBIN,t0,t1,'PCTBIN')
        MISES_LRT[i,j,:] = compute_mise_list(models, t0,t1,'LRT')
        MISES_LRT_clustering[i,j,:] = compute_mise_list(models_clustering,t0,t1,'LRT')

## Figures

In [ ]:
def plot_MISES_given_t0_t1(MISES_LRT,MISES_LRT_clustering,MISES_PCTBIN,i,j,t0s,jumps): 
    
    # a. unpack
    t0 = t0s[i]
    t1 = t0+jumps[j]
    
    num_leafs_list = [np.mean(model.num_leafs[t0:t1]) for model in models]
    num_leafs_list_clust = [np.mean(model.num_leafs[t0:t1]) for model in models_clustering]
    num_bins_list = [model.num_bins for model in models_PCTBIN]

    # b. figure
    fig,ax = figfuns.new() 
    
    ax.semilogx(num_bins_list,MISES_PCTBIN[i,j], marker='x', label='PCTBIN')
    ax.semilogx(num_leafs_list,MISES_LRT[i,j], marker='o', label='LRT')
    ax.semilogx(num_leafs_list_clust,MISES_LRT_clustering[i,j],marker='*',label='LRT (no leads/lags)')
    
    ax.set_xlabel('log number of groups'); 
    ax.set_ylabel(f'mean integrated squared error ($t_0={t0}$, $t_1={t1}$)')
    
    ax.set_xticks(num_bins_list + [np.max(num_leafs_list)])
    ax.set_xticklabels([f'{l:3.0f}' for l in num_bins_list + [max(num_leafs_list)]])
    ax.legend()
    
    fig.tight_layout()    
    figfuns.save(fig,ax,name=f'mise_LRT_vs_PCTBIN_t0{t0}_t1{t1}')
    plt.close()

In [ ]:
for i,t0 in enumerate(t0s): 
    for j,jump in enumerate(jumps): 
        plot_MISES_given_t0_t1(MISES_LRT,MISES_LRT_clustering,MISES_PCTBIN,i,j,t0s,jumps)